In [8]:
import json
import numpy as np
# data preparation
file_path = r"C:\Users\ningn\Downloads\creator.json"
with open(file_path, 'r') as file:
    creator_data = json.load(file)
file_path = r"C:\Users\ningn\Downloads\content.json"
with open(file_path, 'r') as file:
    content_data = json.load(file)

In [9]:
#Ning's EDA plots and ranking

#compute the average of interaction data for scaling
def compute_intersection_average(data, inter_name):
    inter_coll=[]
    for i in data:
        inter = i.get(inter_name, "")
        if inter == "":
            print(i.get("note_id", "")+ " has no "+inter_name)
        else:
            inter_coll.append(int(inter))
    return np.mean(inter_coll)
mean_like = compute_intersection_average(content_data, "liked_count")
mean_coll = compute_intersection_average(content_data, "collected_count")
mean_comment = compute_intersection_average(content_data, "comment_count")
mean_share = compute_intersection_average(content_data, "share_count")
mean_fans = compute_intersection_average(creator_data,"fans")
